# Análise de Dados: Letterboxd

## Sobre o Dataset
A análise foi possível por meio de dois jobs no AWS Glue:

- **Job 1 – Renomear e Droppar:** Feito por Visual ETL. As colunas consideradas irrelevantes para a análise foram removidas (ex.: `cast` – elenco, `release_year` – ano de lançamento, que infelizmente estava vazio, `director` – diretor do filme, etc.).

- **Job 2 – Strings para Listas (countries):** Feito em script PySpark. Aqui foi consideravelmente mais difícil. A coluna `countries` trazia uma string com o nome dos países. Esse job foi criado com o intuito de transformar essas strings em um array de strings, que denominei `country`.

## Sobre a Análise
Dadas as limitações do dataset e a falta de informações sobre filmes brasileiros, que era meu objetivo principal de análise, foi necessário alterar o foco da análise.


In [ ]:
import pandas as pd
import matplotlib as plt

In [ ]:
#Carregando os dados
arquivo = "processed_data_final.parquet"
df = pd.read_parquet(arquivo)

In [ ]:
#Checagens básicas
print(df.head())

In [ ]:
df.shape

In [ ]:
print(df.describe())

In [ ]:
df.info()

# Contagem por País e Top 10

In [ ]:
filmes_por_pais = (
    df["country"]
    .dropna()
    .str.strip()           
    .replace("", pd.NA) #removendo strings vazias
    .value_counts()
)

TOP PAÍSES COM MAIS FILMES NA BASE

In [ ]:
ax = filmes_por_pais.head(10).sort_values(ascending=True).plot(
    kind="barh", figsize=(8,5), color="skyblue", edgecolor="black"
)

ax.set_title("Top 10 países com mais filmes", fontsize=14, weight="bold")
ax.set_xlabel("Quantidade de filmes", fontsize=12)
ax.set_ylabel("País", fontsize=12)
ax.grid(axis="x", linestyle="--", alpha=0.7)

POPULARIDADE POR PAÍS (QUANTOS "WATCHES", OU "ASSISTIDOS")

In [ ]:
ax1 =(
    df.groupby("country")["watches"].sum().sort_values(ascending=False)
    .head(10)
    .plot(kind="barh", figsize=(10,6), color="skyblue", edgecolor="black")
    
    )


Países mais eficientes: filmes com mais assistidas em relação ao total produzido

In [ ]:
filmes = df.groupby("country")["film_title"].nunique() #agrupando por país e contando 
watches = df.groupby("country")["watches"].sum() #somando o total de watches por país
eficiencia = (watches / filmes).sort_values(ascending=False) #dividindo as series (soma de watches por país/filmes únicos por país)


In [ ]:
ax2 = (
    eficiencia.head(10)
    .sort_values(ascending=True)  
    .plot(kind="barh", figsize=(10,6), color="mediumseagreen", edgecolor="black")
)

ax2.set_title("Top 10 países com maior eficiência (watches por filme)", fontsize=14, weight="bold")
ax2.set_xlabel("Média de assistidas por filme", fontsize=12)
ax2.set_ylabel("País", fontsize=12)
ax2.grid(axis="x", linestyle="--", alpha=0.7)

# Adiciona os valores no final das barras
for container in ax2.containers:
    ax2.bar_label(container, fmt='%.0f', fontsize=9)


Acima podemos ver o impacto da trilogia Senhor dos Anéis! Nova Zelândia é um país com relativa baixa produção audiovisual, se comparado aos países de maior tradição cinematográfica, porém, com os 3 filmes da trilogia "O Senhor dos Anéis", sucessos de bilheteria e que até hoje têm relevância cultural, vemos o impacto que isso causa na "eficiência" desse país.

Logo abaixo da Nova Zelândia, temos o Brasil! 

Vamos investigar:

In [ ]:
df_brasil = df[df["country"] == "Brazil"]
df_brasil["film_title"].nunique()

In [ ]:
df_brasil["watches"].sum()

In [ ]:
filmes_brasil= df_brasil.groupby("film_title")["watches"].sum().sort_values(ascending=False)


Desses filmes listados, apenas Bacurau e Cidade de Deus (City of God) são realmente brasileiros.

Vamos avaliar então todos esses 40 "filmes brasileiros"

In [ ]:
print(filmes_brasil)

Realmente, desses filmes alguns não são realmente brasileiros. O fator coprodução, e também que a base de dados considera como country a origem de todas as produtoras envolvidas na produção do filme causam esse "erro".

Identifiquei os filmes realmente brasileiros:

In [ ]:
filmes_brasil_confirmados = [
    "Kiss of the Spider Woman",
    "City of God",
    "360",
    "Trash",
    "Killer Fish",
    "Operações Especiais",
    "Memories They Told Me",
    "Bixa Travesty",
    "The Cambridge Squatter",
    "Beduino",
    "The Lady from the Shanghai Cinema",
    "The Happy Cricket",
    "Baron Olavo, The Horrible",
    "This Night I'll Possess Your Corpse",
    "Bacurau",
    "Medusa"
]

In [ ]:
df_filmes_brasil = df[df["film_title"].isin(filmes_brasil_confirmados)]
print(df_filmes_brasil["film_title"].unique())

In [ ]:
print(
    df_filmes_brasil.groupby("film_title")["watches"]
    .sum()
    .sort_values(ascending=False)
)

Vamos comparar os filmes verdadeiramente brasileiros com os não brasileiros, mas que foram identificados como tal.

In [ ]:
# Soma dos "supostos brasileiros" (df_brasil original)
total_brasil_nao_filtrado = df_brasil["watches"].sum()

# Soma dos confirmados brasileiros
total_brasil_confirmado = df[df["film_title"].isin(filmes_brasil_confirmados)]["watches"].sum()

print("Total Brasil (não filtrado):", total_brasil_nao_filtrado)
print("Total Brasil (confirmados):", total_brasil_confirmado)

print("Diferença:", total_brasil_nao_filtrado - total_brasil_confirmado)


In [ ]:
import matplotlib.pyplot as plt

total_brasil_nao_filtrado = 97962820  
total_brasil_confirmado = 3666333 

# Dados para o gráfico
labels = ["Brasil (não filtrado)", "Brasil (confirmado)"]
values = [total_brasil_nao_filtrado, total_brasil_confirmado]

plt.figure(figsize=(7,5))
plt.bar(labels, values)
plt.title("Comparação: Soma de Watches\nFiltrado vs Confirmado (Brasil)")
plt.ylabel("Total de Watches")
plt.show()

Bom, descobrimos que o Brasil está muito pouco representado nessa base.

# Uma curiosidade é, que produtora brasileira está envolvida em tantos filmes internacionais?

Fazendo uma pesquisa, descobri que se trata da RT Features, produtora brasileira, envolvida em alguns clássicos modernos como "Call Me By Your Name" (2017), filme de Luca Guadagnino, uma co-produção EUA-Itália-Brasil, vencedor de Oscar, "The Lighthouse", sucesso de terror com Willem Dafoe e Robert Pattinson e "Frances Ha", um dos filmes que colocou Greta Gerwig, diretora de Barbie, no "radar" de Hollywood.

De qualquer forma, para mim foi muito interessante descobrir que uma produtora brasileira é tão ativa em tantos sucessos internacionais, em público e crítica.

Meu objetivo inicial era a investigação dos dados sobre filmes brasileiros, mas fui surpreendido negativamente pela pobreza do dataset nesse quesito.

Portanto, seguirei uma nova abordagem. Vou analisar o que consigo dentro desse dataset.

# TOP 20 FILMES MAIS POPULARES

In [ ]:
# Unificando os filmes pelo título, somando os watches
df_filmes = (df.groupby("film_title", as_index=False)
               .agg({"watches": "sum"}))

print(df_filmes.sort_values("watches", ascending=False).head(10))


In [ ]:
# Top 20 outliers (mais simples)
outliers = df_filmes.sort_values("watches", ascending=False).head(20)
print(outliers)

Agora tudo faz sentido!

Era Call Me By Your Name que "puxava" os números do Brasil para cima!

# Correlação Duração x Watches

In [ ]:
# Unificando filmes (um título = um registro)
df_filmes = (df.groupby("film_title", as_index=False)
               .agg({
                   "watches": "sum",   # soma global de visualizações
                   "runtime": "mean"   # duração média (se repetida em países)
               }))

# Removendo linhas sem duração válida
df_filmes = df_filmes.dropna(subset=["runtime"])


In [ ]:
# Correlação de Pearson
correlacao = df_filmes[["runtime", "watches"]].corr(method="pearson")
print(correlacao)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
plt.scatter(df_filmes["runtime"], df_filmes["watches"], alpha=0.5)
plt.title("Correlação: Duração do Filme vs. Popularidade (Watches)")
plt.xlabel("Duração (min)")
plt.ylabel("Total de Watches")
plt.show()


In [ ]:
import numpy as np

x = df_filmes["runtime"]
y = df_filmes["watches"]

m, b = np.polyfit(x, y, 1) 
plt.figure(figsize=(8,6))
plt.scatter(x, y, alpha=0.5, label="Filmes")
plt.plot(x, m*x + b, color="red", label="Tendência")
plt.title("Duração x Popularidade")
plt.xlabel("Duração (min)")
plt.ylabel("Watches")
plt.legend()
plt.show()


Aqui faço um recorte tanto dos TOP 10

In [ ]:
# Definindo o recorte (top 10% por watches)
corte = df_filmes["watches"].quantile(0.90)

# Filtrar só os mais assistidos
top_filmes = df_filmes[df_filmes["watches"] >= corte]


In [ ]:
runtime_mais_comum_top = top_filmes["runtime"].mode()[0]
print("Runtime mais comum entre os top filmes:", runtime_mais_comum_top)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
plt.scatter(df_filmes["runtime"], df_filmes["watches"], alpha=0.5, label="Todos os filmes")

# linha vertical só para os top
plt.axvline(runtime_mais_comum_top, color="red", linestyle="--",
            label=f"Runtime mais comum (Top filmes): {runtime_mais_comum_top} min")

plt.title("Duração do Filme vs. Popularidade (Watches)")
plt.xlabel("Duração (min)")
plt.ylabel("Total de Watches")
plt.legend()
plt.show()


RUNTIME MAIS PLAUSÍVEL

In [ ]:
# Filtrar filmes de até 210 minutos (3h30)
df_filmes_filtrado = df_filmes[df_filmes["runtime"].between(60, 210)]

# Correlação só nesse intervalo
correlacao_filtrada = df_filmes_filtrado[["runtime", "watches"]].corr(method="pearson")
print(correlacao_filtrada)


In [ ]:
import matplotlib.pyplot as plt

# Dataset original
x_all = df_filmes["runtime"]
y_all = df_filmes["watches"]

# Dataset filtrado (até 210 min)
df_filmes_filtrado = df_filmes[df_filmes["runtime"].between(0, 210)]
x_filtrado = df_filmes_filtrado["runtime"]
y_filtrado = df_filmes_filtrado["watches"]

plt.figure(figsize=(14,6))

# Scatter completo
plt.subplot(1,2,1)
plt.scatter(x_all, y_all, alpha=0.4)
plt.title("Runtime vs Watches (Todos os filmes)")
plt.xlabel("Runtime (min)")
plt.ylabel("Watches")

# Scatter filtrado
plt.subplot(1,2,2)
plt.scatter(x_filtrado, y_filtrado, alpha=0.4, color="green")
plt.title("Runtime vs Watches (0–210 min)")
plt.xlabel("Runtime (min)")
plt.ylabel("Watches")

plt.tight_layout()
plt.show()
